In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [2]:
wuenic = "/Users/shilpa/Desktop/Desktop/RICE_DATA/Global_Vaccination_Progress/Shilpa/Plots/Clean_Plots/IPV/df_IPV1_All_countries.csv"
IPV1 = pd.read_csv(wuenic)
IPV1.head(5)
df_IPV1= IPV1.sort_values(['year'], ascending= True)
df_IPV1

,Unnamed: 0,Unnamed: 0.1,0,Unnamed: 0.1.1,Index,country,year,Vaccine,Subgroup,Coverage,Vaccinated,Target,values
0,0,132,132,203,2015IPV1Albania,Albania,2015,IPV1,CEECIS,99,35000,35000,3148281
1032,9,1210,1210,4119,2015IPV1Botswana,Botswana,2015,IPV1,ESAR,8,4000,55000,1540425
1031,769,8568,8568,34561,2015IPV1Vanuatu,Vanuatu,2015,IPV1,EAPR,4,500,8000,219472
1030,1,220,220,411,2015IPV1Algeria,Algeria,2015,IPV1,MENA,8,79000,991000,29742979
1029,2,330,330,939,2015IPV1Antigua and Barbuda,Antigua and Barbuda,2015,IPV1,LACR,21,500,1000,71704
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6858,8599,8491,8491,34338,2018IPV1Uzbekistan,Uzbekistan,2018,IPV1,CEECIS,56,382000,682000,32955400
6859,8592,7963,7963,32065,2018IPV1Trinidad and Tobago,Trinidad and Tobago,2018,IPV1,LACR,99,17000,18000,1389858
6860,8582,7303,7303,29738,2018IPV1South Sudan,South Sudan,2018,IPV1,ESAR,34,123000,362000,10975920
6842,8562,5873,5873,23431,2018IPV1Pakistan,Pakistan,2018,IPV1,ROSA,75,4226000,5634000,212215030


In [3]:
df_IPV1.to_csv('../Plots/df_IPV1_All_countries.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Plots/df_IPV1_All_countries.csv'

In [4]:
dataset = df_IPV1
x_column = 'Target'
y_column = 'Coverage'
size_column = 'values'
bubble_column = 'country'
time_column = 'year'


In [5]:
# Get the years in the dataset
years = dataset[time_column].unique()

# Make the grid
grid = pd.DataFrame()
col_name_template = '{year}+{header}_grid'
for year in years:
    dataset_by_year = dataset[(dataset['year'] == int(year))]
    for col_name in [x_column, y_column, bubble_column]:
        # Each column name is unique
        temp = col_name_template.format(
            year=year, header=col_name
        )
        if dataset_by_year[col_name].size != 0:
            grid = grid.append({'value': list(dataset_by_year[col_name]), 'key': temp}, 
                               ignore_index=True)

grid.sample(5)

,key,value
8,2017+country_grid,"[Afghanistan, Albania, Algeria, Antigua and Ba..."
5,2016+country_grid,"[Benin, Vanuatu, Bahrain, Equatorial Guinea, B..."
2,2015+country_grid,"[Albania, Botswana, Vanuatu, Algeria, Antigua ..."
11,2018+country_grid,"[Barbados, Algeria, Bosnia and Herzegovina, Al..."
0,2015+Target_grid,"[35000, 55000, 8000, 991000, 1000, 21000, 2906..."


In [6]:
# Define figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# Get the earliest year
year = min(years)

# Make the trace
trace = {
    'x': grid.loc[grid['key']==col_name_template.format(
        year=year, header=x_column
    ), 'value'].values[0], 
    'y': grid.loc[grid['key']==col_name_template.format(
        year=year, header=y_column
    ), 'value'].values[0],
    'mode': 'markers',
    'text': grid.loc[grid['key']==col_name_template.format(
        year=year, header=bubble_column
    ), 'value'].values[0]
}
# Append the trace to the figure
figure['data'].append(trace)

# Plot the figure
plotly.offline.iplot(figure, config={'scrollzoom': True})


//anaconda3/lib/python3.7/site-packages/plotly/offline/offline.py:156: UserWarning:


Unrecognized config options supplied: ['scrollzoom']



In [7]:
# Get the max and min range of both axes
xmin = min(np.log10(dataset[x_column]))*0.98
xmax = max(np.log10(dataset[x_column]))*1.02
ymin = min(dataset[y_column])*0.75
ymax = max(dataset[y_column])*1.25

# Modify the layout
figure['layout']['xaxis'] = {'title': 'Target', 'type': 'log', 
                             'range': [xmin, xmax]}   
figure['layout']['yaxis'] = {'title': '%Vaccinated', 
                             'range': [ymin, ymax]} 
figure['layout']['title'] = 'Global IPV1 Vaccination'
figure['layout']['showlegend'] = False
figure['layout']['hovermode'] = 'closest'
plotly.offline.iplot(figure, config={'scrollzoom': True})

In [8]:
for year in years:
    # Make a frame for each year
    frame = {'data': [], 'name': str(year)}
    
    # Make a trace for each frame
    trace = {
        'x': grid.loc[grid['key']==col_name_template.format(
            year=year, header=x_column
        ), 'value'].values[0],
        'y': grid.loc[grid['key']==col_name_template.format(
            year=year, header=y_column
        ), 'value'].values[0],
        'mode': 'markers',
        'text': grid.loc[grid['key']==col_name_template.format(
            year=year, header=bubble_column
        ), 'value'].values[0],
        'type': 'scatter'
    }
    # Add trace to the frame
    frame['data'].append(trace)
    # Add frame to the figure
    figure['frames'].append(frame) 

plotly.offline.iplot(figure, config={'scrollzoom': True})

In [9]:
figure['layout']['sliders'] = {
    'args': [
        'slider.value', {
            'duration': 400,
            'ease': 'cubic-in-out'
        }
    ],
    'initialValue': min(years),
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

from bubbly.bubbly import add_slider_steps

for year in years:
    add_slider_steps(sliders_dict, year)
    
figure['layout']['sliders'] = [sliders_dict]
plotly.offline.iplot(figure, config={'scrollzoom': True})

In [10]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 
                                                             'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration':0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
    ]
plotly.offline.iplot(figure, config={'scrollzoom': True})

In [11]:
from bubbly.bubbly import make_grid, set_layout, get_trace

# Define the new variable
size_column = 'values' 

# Make the grid
years = dataset[time_column].unique()

col_name_template = '{}+{}_grid'
column_names = [x_column, y_column, bubble_column, size_column]
grid = make_grid(dataset, column_names, time_column, years)
    
# Set the layout
figure, sliders_dict = set_layout(x_title='Target', y_title='Vaccinated', 
            title='IPV1 Vaccination', x_logscale=True, y_logscale=False, 
            show_slider=True, slider_scale=years, show_button=True, show_legend=False, 
            height=650)
# Add the base frame
year = min(years)
col_name_template_year = col_name_template.format(year, {})
trace = get_trace(grid=grid, col_name_template=col_name_template_year, 
                    x_column=x_column, y_column=y_column, 
                    bubble_column=bubble_column, size_column=size_column)
figure['data'].append(trace)


# Add time frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    col_name_template_year = col_name_template.format(year, {})
    trace = get_trace(grid=grid, col_name_template=col_name_template_year, 
                    x_column=x_column, y_column=y_column, 
                    bubble_column=bubble_column, size_column=size_column)
    frame['data'].append(trace)
    figure['frames'].append(frame) 
    add_slider_steps(sliders_dict, year)

# Set the layout once more
figure['layout']['xaxis']['autorange'] = True
figure['layout']['yaxis']['range'] = [20, 200]
figure['layout']['sliders'] = [sliders_dict]

#print(frame)
# Plot the animation
plotly.offline.iplot(figure, config={'scrollzoom': True})